In [100]:
import pandas as pd
from bnlp.corpus import stopwords, punctuations, digits
from bnlp import BasicTokenizer
from transformers import BertModel, BertTokenizer, BertForSequenceClassification

In [101]:
threshold = .7

In [102]:
train_set = pd.read_csv("train_set_2.csv")
test_set = pd.read_csv("test_set_2.csv")
validation_set = pd.read_csv("test_set_1.csv")

In [103]:
print(train_set)
print(len(train_set))

                                                   text  label
0     মামা ভয়ে ভয়ে লুকিং গ্লাসের কাপড় সরিয়ে দিয়...      4
1     এই বয়সে নাতি নাতনিদের সঙ্গে আনন্দে মেতে থাকার ...      3
2                         আপু আপনাকে খুব সুন্দর লাগছে        0
3     তোমার শহরের কোণে কেউ মায়া জমায় কি  আমি এখনো ভা...      1
4     এর মাথা পুরোটাই নষ্ট  পশ্চিম বঙ্গবাসী একে ভোট ...      2
...                                                 ...    ...
3715  আমি কষ্টের স্মৃতি নিয়ে করি বসবাস আমায় নিয়ে কখন...      1
3716  আজ থেকে অফিসে এবং বাসায় প্রথম আলো পত্রিকা  বাদ...      2
3717  এই নোয়াখাইল্লা বাইনচুদেরে বেধে লেংটা করে রোদে ...      2
3718  হিরু আলম জায়েদ খান  ওরা ও  নায়েক আর নোবেল ও এক...      3
3719  সত্যি সাকিব ভাই আজ আপনি আবার প্রমান করলেন কেন ...      0

[3720 rows x 2 columns]
3720


In [104]:
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [105]:
def filters(corpus):
    res = []
    for i in corpus:
        temp_i = [*i]
        flag = 0
        for j in stopwords:
#             print(f"i={temp_i} j={[*j]} jac={jaccard(temp_i,[*j])}")
            if jaccard(temp_i,j)>threshold:
                flag = 1
                break
        if(flag == 1):
            continue
            
        if i in stopwords:
            continue

        if i in punctuations + '‘' + '’':
            continue
        if i in digits:
            continue

        res.append(i)

    return res

In [106]:
tokenizer = BasicTokenizer()
tokenizer_bert = BertTokenizer.from_pretrained("sagorsarker/bangla-bert-base")

In [107]:
train_set_text_list = train_set.text.tolist()
train_set_label_list = train_set.label.tolist()
print(len(train_set_text_list))
print(len(train_set_label_list))

3720
3720


In [108]:
new_train_set_text_list = []
for i in train_set_text_list:
    temp_list = tokenizer.tokenize(i)
    new_train_set_text_list.append(filters(temp_list))
# print(new_train_set_text_list)
    

In [109]:
print(len(new_train_set_text_list))

3720


In [110]:
final_list = []
for i in new_train_set_text_list:
    final_list.append(" ".join(i))
# print(final_list)

In [111]:
for i in range(len(final_list)):
    if type(final_list[i])!=str:
        print(i)
    if len(final_list[i])==0:
        print(i)

32
1760
2640


In [112]:
for i in range(len(final_list)):
    if type(final_list[i])!=str:
        print(i)
    if len(final_list[i])==0:
        final_list[i]=train_set_text_list[i]

In [113]:
for i in range(len(final_list)):
    if type(final_list[i])!=str:
        print(i)
    if len(final_list[i])==0:
        print(i)

In [114]:
write_csv = pd.DataFrame(list(zip(final_list, train_set_label_list)),
               columns =['text', 'label'])

In [115]:
write_csv.to_csv("stop_words_removed_train_set_2.csv",index=False)